In [ ]:
import torch
from torch import nn
import numpy as np
torch.cuda.empty_cache()
import torch.nn.functional as F
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


import gc
#del y,prt
gc.collect()

torch.no_grad()

In [1]:
from B_VAE.B_VAE import b_encodeco
from Train_utils import train_utils
from B_VAE.Utils_VAE import loss_fn
from Train_utils.train_utils import train,test,K_fold_train

import torch

device='cuda'

import sys
import os
sys.path.append(os.path.join("..","Data_prep"))
from torchvision import transforms
from Custom_dataloader import *
from Transforms import phantom_segmentation
from Transforms import multi_image_resize
from Transforms import multi_ToTensor
from Transforms import output_transform

import matplotlib.pyplot as plt
import torch

DB="/run/user/1000/gvfs/afp-volume:host=MyCloudPR4100.local,user=paltas,volume=Paltas_DataBase/Data_Base_v2"
#DB="//MYCLOUDPR4100/Paltas_DataBase/Data_Base_v2"
d_tt=transforms.Compose([
    phantom_segmentation(False),
    multi_image_resize(ImType=['PhantomRGB'],size=(500,500)),
    multi_ToTensor(ImType=['PhantomRGB']),
    output_transform()
    ])
#datar=Dataset_direct(root_dir=DB,ImType=['PhantomRGB','SenteraRGB'],Intersec=False)
#datab=Dataset_direct(root_dir=DB,ImType=['PhantomRGB'],Intersec=False,transform=d_tt)


In [2]:
datab=Dataset_direct(root_dir=DB,ImType=['PhantomRGB'],Intersec=False,transform=d_tt)

train_s=int((len(datab))*0.7)
test_s=int(len(datab)-train_s)
train_set, test_set = torch.utils.data.random_split(datab, [train_s, test_s])

dataloader_train=torch.utils.data.DataLoader(train_set,batch_size=5,shuffle=True)
dataloader_test=torch.utils.data.DataLoader(test_set,batch_size=5,shuffle=True)

In [3]:
model=b_encodeco(image_dim=int(500),
                 image_channels=3,
                 repr_sizes=[32,64,256],
                 layer_sizes=[],
                 latent_space_size=10,
                device=device)
model.to(device)

#opt=torch.optim.Adam(model.parameters(),lr=1e-3)

b_encodeco(
  (encoder_conv): b_encoder_conv(
    (im_layers): ModuleList(
      (0): s_conv(
        (Conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (act): ReLU()
      )
      (1): s_conv(
        (Conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (act): ReLU()
      )
      (2): s_conv(
        (Conv): Conv2d(64, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (act): ReLU()
      )
    )
  )
  (encoder_NN_mu): NeuralNet(
    (layers): ModuleList(
      (0): Sequential(
        (0): Linear(in_features=1000000, out_features=10, bias=True)
        (1): ReLU()
      )
    )
  )
  (encoder_NN_sig): NeuralNet(
    (layers): ModuleList(
      (0): Sequential(
        (0): Linear(in_features=1000000, out_features=10, bias=True)
        (1): ReLU()
      )
    )
  )
  (flatten): s_view()
  (decoder_NN): NeuralNet(
    (layers): ModuleList(
      (0): Sequential(
        (0): Linear(in_features=10, out_features=10

In [5]:
import pathlib
from Train_utils.train_utils import train,test,train_test,K_fold_train

In [ ]:
opt=torch.optim.Adam(model.parameters(),lr=1e-3)
train(model=model,
      optimizer=opt,
     dataloader=dataloader_test,
     use_cuda=False,
     loss_function=loss_fn)

instances: 0it [00:00, ?it/s]

iniciio
model result
model loss
zero grad


/home/lambda/paltas/lib/python3.6/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


loss back


instances: 1it [01:08, 68.93s/it]

step
total loss 10876896.0000	 BCE 10876896.0000	 KLD 0.0002


In [ ]:
opt=torch.optim.Adam(model.parameters(),lr=1e-3)
test(model=model,
     dataloader=dataloader_test,
     use_cuda=False,
     loss_function=loss_fn)

In [ ]:
train_test(model=model,
           optimizer=opt,
           dataloader_train=dataloader_train,
           dataloader_test=dataloader_test,
           use_cuda=False,
           loss_function=loss_fn,
           epochs=2,
           data_train_dir=str(pathlib.Path().absolute()))

In [6]:
K_fold_train(model=model,
                dataset=datab,
                epochs=100,
                batch_size=2,
                use_cuda=True,
                folds=5,
                data_train_dir=str(pathlib.Path().absolute()),
                loss_fn=loss_fn
     )

Epoch:   0%|          | 0/100 [00:00<?, ?it/s]

instances: 0it [01:23, ?it/s]
folds:   0%|          | 0/5 [01:29<?, ?it/s]


RuntimeError: mat1 and mat2 shapes cannot be multiplied (5x1016064 and 1000000x10)